## Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
from IPython.display import display
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor
import matplotlib.pyplot as plt

## Load dataset

In [ ]:
path = "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/dataframe_tabpfn5"
df = pd.read_parquet(path)

# filter once outside columns loop since condition is independent of target column
filtered_df = df[(df['fr_eng'] > (10/60))]
filtered_df = filtered_df.sort_values(by='time')

## Mean imputer

In [ ]:
from sklearn.impute import SimpleImputer

# Choose a variable as y and other variables as X
target_column_list = ['te_air_scav_rec', 'pr_baro', 'pd_air_ic__0', 'pr_exh_rec', 'pr_air_scav_ecs', 're_eng_load']
column_dfs = []

for column in target_column_list:
    

    # Drop rows with NaN target value
    if column not in filtered_df.columns:
        print(f"  Skipping {column} in current df: column missing.")
        continue

    df_col = filtered_df.dropna(subset=[column])
    if df_col.empty:
        print(f"  Skipping {column} in current df: no data after dropna.")
        continue

    column_dfs.append(df_col)


if not column_dfs:
    raise ValueError("No valid dataframes to concatenate. Check filtering conditions or target columns.")
    
combined_df = pd.concat(column_dfs, ignore_index=True)
df_length = len(combined_df)

# Train-test split
train_df = combined_df.iloc[:int(df_length * 0.8)]
test_df = combined_df.iloc[int(df_length * 0.8):]

# Drop non-numeric columns
numeric_cols = train_df.select_dtypes(include='number').columns
numeric_cols = [col for col in numeric_cols if train_df[col].notnull().any()]

train_numeric = train_df[numeric_cols]
test_numeric = test_df[numeric_cols]
test_copy = test_numeric.copy()

imputer = SimpleImputer(strategy='mean')
imputer.fit(train_numeric)

test_imputed_numeric = pd.DataFrame(
    imputer.transform(test_copy),
    columns=train_numeric.columns,
    index=test_copy.index
)

# Evaluate R² for each target column
for column in target_column_list:
    print(f"\n==== Processing target column: {column} ====")
    if column not in train_numeric.columns:
        print(f"Skipping {column}: not numeric or no non-missing values in train")
        continue

    non_missing_idx = test_copy[test_copy[column].notnull()].index
    mask_size = min(20, len(non_missing_idx))
    mask_idx = np.random.choice(non_missing_idx, size=mask_size, replace=False)

    true_values = test_copy.loc[mask_idx, column].copy()
    test_copy.loc[mask_idx, column] = np.nan

    # Impute again for evaluation
    test_imputed_numeric = pd.DataFrame(
        imputer.transform(test_copy),
        columns=train_numeric.columns,
        index=test_copy.index
    )

    imputed_values = test_imputed_numeric.loc[mask_idx, column]

    # Compute R²
    r2 = r2_score(true_values, imputed_values)

    # Evaluate the model
    mse = mean_squared_error(true_values, imputed_values)
    mae = mean_absolute_error(true_values, imputed_values)

    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R² Score:", r2)

    # Create figure and axes
    fig = plt.figure(figsize=(8, 10))
    grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.5)

    main_ax = fig.add_subplot(grid[1:4, 0:3])
    x_hist = fig.add_subplot(grid[0, 0:3], sharex=main_ax)
    y_hist = fig.add_subplot(grid[1:4, 3], sharey=main_ax)

    # Scatter plot with R²
    main_ax.scatter(true_values, imputed_values, alpha=0.6)
    main_ax.set_xlabel('Original Data')
    main_ax.set_ylabel('Predicted Data')
    main_ax.set_title(f'R² Plot (R² = {r2:.2f})')

    # Add y = x line
    min_val = min(min(true_values), min(imputed_values))
    max_val = max(max(true_values), max(imputed_values))
    main_ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    main_ax.legend()

    # Histograms
    x_hist.hist(true_values, bins=20, alpha=0.7)

    y_hist.hist(imputed_values, bins=20, orientation='horizontal', alpha=0.7)

    # Show the plot
    plt.show() 

## Linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer

df = filtered_df.copy()
df = df.sort_values(by='time')

# Collect all valid rows for each target column
column_dfs = []
for column in target_column_list:
    if column not in df.columns:
        print(f"Skipping {column}: column missing")
        continue
    df_col = df.dropna(subset=[column])  # drop rows where target is NaN
    if df_col.empty:
        print(f"Skipping {column}: no data after dropna")
        continue
    column_dfs.append(df_col)

if not column_dfs:
    raise ValueError("No valid dataframes to concatenate.")

combined_df = pd.concat(column_dfs, ignore_index=True)

# === Train-test split ===
df_length = len(combined_df)
train_df = combined_df.iloc[:int(df_length * 0.8)].copy()
test_df = combined_df.iloc[int(df_length * 0.8):].copy()

# === Numeric columns only, drop 'time' if present ===
numeric_cols = train_df.select_dtypes(include='number').columns
numeric_cols = [col for col in numeric_cols if col != 'time']

# Drop columns that are entirely NaN
train_numeric = train_numeric.dropna(axis=1, how="all")
test_numeric = test_numeric[train_numeric.columns]

# === Apply KNN imputer ===
imputer = KNNImputer(n_neighbors=5)

train_imputed = pd.DataFrame(
    imputer.fit_transform(train_numeric),
    columns=train_numeric.columns,
    index=train_numeric.index
)

test_imputed = pd.DataFrame(
    imputer.transform(test_numeric),
    columns=test_numeric.columns,
    index=test_numeric.index
)

# === Linear Regression for each target column ===
for column in target_column_list:
    if column not in train_imputed.columns:
        print(f"Skipping {column}: not numeric or dropped")
        continue

    print(f"\n==== Linear Regression for target: {column} ====")

    # Features and target
    X_train = train_imputed.drop(columns=[column])
    y_train = train_imputed[column]

    X_test = test_imputed.drop(columns=[column])
    y_test = test_imputed[column]

    # Fit model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict
    predictions = model.predict(X_test)

    # Calculate R² score
    r2 = r2_score(y_test, predictions)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R² Score:", r2)

    # Create figure and axes
    fig = plt.figure(figsize=(8, 8))
    grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.5)

    main_ax = fig.add_subplot(grid[1:4, 0:3])
    x_hist = fig.add_subplot(grid[0, 0:3], sharex=main_ax)
    y_hist = fig.add_subplot(grid[1:4, 3], sharey=main_ax)

    # Scatter plot with R²
    main_ax.scatter(y_test, predictions, alpha=0.6)
    main_ax.set_xlabel('Original Data')
    main_ax.set_ylabel('Predicted Data')
    main_ax.set_title(f'R² Plot (R² = {r2:.2f})')

    # Add y = x line
    min_val = min(min(y_test), min(predictions))
    max_val = max(max(y_test), max(predictions))
    main_ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    main_ax.legend()

    # Histograms
    x_hist.hist(y_test, bins=20, alpha=0.7)

    y_hist.hist(predictions, bins=20, orientation='horizontal', alpha=0.7)

    # Show the plot
    plt.show()

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

df = filtered_df.copy()
df = df.sort_values(by='time')

# Collect all valid rows for each target column
column_dfs = []
for column in target_column_list:
    if column not in df.columns:
        print(f"Skipping {column}: column missing")
        continue
    df_col = df.dropna(subset=[column])  # drop rows where target is NaN
    if df_col.empty:
        print(f"Skipping {column}: no data after dropna")
        continue
    column_dfs.append(df_col)

if not column_dfs:
    raise ValueError("No valid dataframes to concatenate.")

combined_df = pd.concat(column_dfs, ignore_index=True)

# === Train-test split ===
df_length = len(combined_df)
train_df = combined_df.iloc[:int(df_length * 0.8)].copy()
test_df = combined_df.iloc[int(df_length * 0.8):].copy()

# === Numeric columns only, drop 'time' if present ===
numeric_cols = train_df.select_dtypes(include='number').columns
numeric_cols = [col for col in numeric_cols if col != 'time']

train_numeric = train_df[numeric_cols].copy()
test_numeric = test_df[numeric_cols].copy()

# Drop columns that are entirely NaN
train_numeric = train_numeric.dropna(axis=1, how="all")
test_numeric = test_numeric[train_numeric.columns]  # align columns

# === Apply KNN imputer ===
imputer = KNNImputer(n_neighbors=5)

train_imputed = pd.DataFrame(
    imputer.fit_transform(train_numeric),
    columns=train_numeric.columns,
    index=train_numeric.index
)

test_imputed = pd.DataFrame(
    imputer.transform(test_numeric),
    columns=test_numeric.columns,
    index=test_numeric.index
)

# === Random Forest for each target column ===
for column in target_column_list:
    if column not in train_imputed.columns:
        print(f"Skipping {column}: not numeric or dropped")
        continue

    print(f"\n==== Random Forest for target: {column} ====")

    # Features and target
    X_train = train_imputed.drop(columns=[column])
    y_train = train_imputed[column]

    X_test = test_imputed.drop(columns=[column])
    y_test = test_imputed[column]

    # Fit model
    model = RandomForestRegressor(
        n_estimators=100, 
        random_state=42, 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Predict
    predictions = model.predict(X_test)

    # Calculate R² score
    r2 = r2_score(y_test, predictions)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R² Score:", r2)

    # Create figure and axes
    fig = plt.figure(figsize=(8, 8))
    grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.5)

    main_ax = fig.add_subplot(grid[1:4, 0:3])
    x_hist = fig.add_subplot(grid[0, 0:3], sharex=main_ax)
    y_hist = fig.add_subplot(grid[1:4, 3], sharey=main_ax)

    # Scatter plot with R²
    main_ax.scatter(y_test, predictions, alpha=0.6)
    main_ax.set_xlabel('Original Data')
    main_ax.set_ylabel('Predicted Data')
    main_ax.set_title(f'R² Plot (R² = {r2:.2f})')

    # Add y = x line
    min_val = min(min(y_test), min(predictions))
    max_val = max(max(y_test), max(predictions))
    main_ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    main_ax.legend()

    # Histograms
    x_hist.hist(y_test, bins=20, alpha=0.7)

    y_hist.hist(predictions, bins=20, orientation='horizontal', alpha=0.7)

    # Show the plot
    plt.show()

## tabPFN

In [ ]:
# Choose a variable as y and other variables as X
target_column_list = ['te_air_scav_rec', 'pr_baro', 'pd_air_ic__0', 'pr_exh_rec', 'pr_air_scav_ecs', 're_eng_load', 'te_seawater', 'bo_aux_blower_running', 'IN_ENGINE_RUNNING_MODE']

for column in target_column_list:
    print(f"\n==== Processing target column: {column} ====")
    column_dfs = []
    
    # filter once outside columns loop since condition is independent of target column
    filtered_df = df[(df['fr_eng'] > (10/60))]
    filtered_df = filtered_df.iloc[:, filtered_df.nunique().values > 1]
    filtered_df = filtered_df.iloc[:, filtered_df.isna().mean().values < 0.95]
    filtered_df = filtered_df.sort_values(by='time')

    # Drop rows with NaN target value
    if column not in filtered_df.columns:
        print(f"  Skipping {column} in current df: column missing.")
        continue

    df_col = filtered_df.dropna(subset=[column])
    if df_col.empty:
        print(f"  Skipping {column} in current df: no data after dropna.")
        continue

    column_dfs.append(df_col)


    if not column_dfs:
        raise ValueError("No valid dataframes to concatenate. Check filtering conditions or target columns.")
    combined_df = pd.concat(column_dfs, ignore_index=True)
    df_length = len(combined_df)

    # Train-test split
    train_df = combined_df.iloc[:int(df_length * 0.8)]
    test_df = combined_df.iloc[int(df_length * 0.8):]

    # Sample (cap at max size)
    train_sample_size = min(len(train_df), 10000)
    test_sample_size = min(len(test_df), 2500)

    train_sampled = train_df.sample(n=train_sample_size, random_state=42).dropna(subset=[column])
    test_sampled = test_df.sample(n=test_sample_size, random_state=42).dropna(subset=[column])

    print(f"Training size for {column}:", len(train_sampled))
    print(f"Testing size for {column}:", len(test_sampled))

    X_train = train_sampled.drop(columns=[column, 'time'])
    y_train = train_sampled[column]
    X_test = test_sampled.drop(columns=[column, 'time'])
    y_test = test_sampled[column]

    # Initialize and train regressor
    regressor = TabPFNRegressor()
    regressor.fit(X_train, y_train)

    # Predict
    predictions = regressor.predict(X_test)

    # Calculate R² score
    r2 = r2_score(y_test, predictions)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R² Score:", r2)

    # Create figure and axes
    fig = plt.figure(figsize=(8, 8))
    grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.5)

    main_ax = fig.add_subplot(grid[1:4, 0:3])
    x_hist = fig.add_subplot(grid[0, 0:3], sharex=main_ax)
    y_hist = fig.add_subplot(grid[1:4, 3], sharey=main_ax)

    # Scatter plot with R²
    main_ax.scatter(y_test, predictions, alpha=0.6)
    main_ax.set_xlabel('Original Data')
    main_ax.set_ylabel('Predicted Data')
    main_ax.set_title(f'R² Plot (R² = {r2:.2f})')

    # Add y = x line
    min_val = min(min(y_test), min(predictions))
    max_val = max(max(y_test), max(predictions))
    main_ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    main_ax.legend()

    # Histograms
    x_hist.hist(y_test, bins=20, alpha=0.7)

    y_hist.hist(predictions, bins=20, orientation='horizontal', alpha=0.7)

    # Show the plot
    plt.show() 